# Metric Presentation and Visualization
## Necessary packages and functions call

- DDPM-TS: Interpretable Diffusion for Time Series Generation
- Metrics: 
    - discriminative_metrics
    - predictive_metrics
    - visualization

In [2]:
%pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import sys
sys.path.append(os.path.join(os.path.dirname('__file__'), '../'))

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

from Utils.metric_utils import display_scores
from Utils.discriminative_metric import discriminative_score_metrics
from Utils.predictive_metric import predictive_score_metrics

## Data Loading

Load original dataset and preprocess the loaded data.

In [4]:
iterations = 5
ori_data = np.load('../toy_exp/samples/sine_ground_truth_24_train.npy')
# ori_data = np.load('../OUTPUT/{dataset_name}/samples/{dataset_name}_norm_truth_{seq_length}_train.npy')  # Uncomment the line if dataset other than Sine is used.
fake_data = np.load('../toy_exp/ddpm_fake_sines.npy')

## Evaluate the generated data

### 1. Discriminative score

To evaluate the classification accuracy between original and synthetic data using post-hoc RNN network. The output is | classification accuracy - 0.5 |.

- metric_iteration: the number of iterations for metric computation.

In [5]:
discriminative_score = []

for i in range(iterations):
    temp_disc, fake_acc, real_acc = discriminative_score_metrics(ori_data[:], fake_data[:ori_data.shape[0]])
    discriminative_score.append(temp_disc)
    print(f'Iter {i}: ', temp_disc, ',', fake_acc, ',', real_acc, '\n')
      
print('sine:')
display_scores(discriminative_score)
print()

AttributeError: `GRUCell` is not available with Keras 3.

## Evaluate the generated data

### 2. Predictive score

To evaluate the prediction performance on train on synthetic, test on real setting. More specifically, we use Post-hoc RNN architecture to predict one-step ahead and report the performance in terms of MAE. 

The model learns to predict the last dimension with one more step.

In [7]:
predictive_score = []
for i in range(iterations):
    temp_pred = predictive_score_metrics(ori_data, fake_data[:ori_data.shape[0]])
    predictive_score.append(temp_pred)
    print(i, ' epoch: ', temp_pred, '\n')
      
print('sine:')
display_scores(predictive_score)
print()

training: 100%|████████████████████████████████████████████████████████████████████| 5000/5000 [04:59<00:00, 16.67it/s]


0  epoch:  0.09314072894950631 



training: 100%|████████████████████████████████████████████████████████████████████| 5000/5000 [04:52<00:00, 17.08it/s]


1  epoch:  0.09352861018187178 



training: 100%|████████████████████████████████████████████████████████████████████| 5000/5000 [05:04<00:00, 16.41it/s]


2  epoch:  0.09322281220011404 



training: 100%|████████████████████████████████████████████████████████████████████| 5000/5000 [04:56<00:00, 16.85it/s]


3  epoch:  0.09313142589665173 



training: 100%|████████████████████████████████████████████████████████████████████| 5000/5000 [05:01<00:00, 16.57it/s]


4  epoch:  0.09342775384532898 

sine:
Final Score:  0.09329026621469458 ± 0.00022198749029314803

